In [59]:
import numpy as np
from six.moves import cPickle as pickle
import tensorflow as tf

pixels_height = 32
pixels_width = 32
num_labels = 11
num_channels = 1 #grayscale

In [60]:
def getDataAndLabel(dataset):
    return dataset['data'], dataset['label']

pickleFile = open('tensorflow_data.pickle', 'r')
tensorflowData = pickle.load(pickleFile)
train_X, train_labels = getDataAndLabel(tensorflowData['train'])
test_X, test_labels = getDataAndLabel(tensorflowData['test'])
valid_X, valid_labels = getDataAndLabel(tensorflowData['valid'])
predict_X, predict_labels = getDataAndLabel(tensorflowData['predict'])


In [61]:
print(len(train_X))
print(len(test_X))
print(len(valid_X))

33402
13068
15000


In [62]:
def reformatData(X):
    return X.reshape(-1, pixels_width, pixels_height, num_channels).astype(np.float32)
   

train_dataset = reformatData(train_X)
valid_dataset = reformatData(valid_X)
test_dataset = reformatData(test_X)
predict_dataset = reformatData(predict_X)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (33402, 32, 32, 1), (33402, 6))
('Validation set', (15000, 32, 32, 1), (15000, 6))
('Test set', (13068, 32, 32, 1), (13068, 6))


In [63]:
batch_size = 64
patch_size = 5

depth_1 = 32
depth_2 = 64
depth_3 = 96
num_hidden = 128

In [64]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [65]:
pixels_width // 4 * pixels_height // 4 * 64

4096

In [94]:
graph = tf.Graph()
with graph.as_default():
    # Input data
    # Load training, testing and validation data into constants that
    # are attached to the graph
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, pixels_width, pixels_height, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 6))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
        
    #Conv1
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth_1], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth_1]))
    
    #Conv2
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, pixels_width, depth_2], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(0.1, shape=[depth_2]), dtype=tf.float32)
    
    #Fully connected 1
    layer3_weights = tf.Variable(tf.truncated_normal([pixels_width // 4 * pixels_height // 4 * 64, 512], stddev=0.1, dtype=tf.float32))
    layer3_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=tf.float32))
    
    #Outputs
    #First classifer
    cls1_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=0.1, dtype=tf.float32))
    cls1_biases = tf.Variable(tf.constant(0.1, shape=[num_labels], dtype=tf.float32))
    
    #Second classifier
    cls2_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=0.1, dtype=tf.float32))
    cls2_biases = tf.Variable(tf.constant(0.1, shape=[num_labels], dtype=tf.float32))
    
    #Third classifier
    cls3_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=0.1, dtype=tf.float32))
    cls3_biases = tf.Variable(tf.constant(0.1, shape=[num_labels], dtype=tf.float32))
    
    #4th classifier
    cls4_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=0.1, dtype=tf.float32))
    cls4_biases = tf.Variable(tf.constant(0.1, shape=[num_labels], dtype=tf.float32))
    
    #5th classifier
    cls5_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=0.1, dtype=tf.float32))
    cls5_biases = tf.Variable(tf.constant(0.1, shape=[num_labels], dtype=tf.float32))
        
    #model
    def model(data, isTrain = False):
        conv = tf.nn.conv2d(data, layer1_weights, strides = [1, 1, 1, 1], padding='SAME')
        
        relu = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding ='SAME')
        
        conv = tf.nn.conv2d(pool, layer2_weights, strides = [1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        shape = pool.get_shape().as_list()
        
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        if isTrain == True:
            hidden = tf.nn.dropout(hidden, 0.7)
        
        logit_cls1 = tf.matmul(hidden, cls1_weights) + cls1_biases
        logit_cls2 = tf.matmul(hidden, cls2_weights) + cls2_biases
        logit_cls3 = tf.matmul(hidden, cls3_weights) + cls3_biases
        logit_cls4 = tf.matmul(hidden, cls4_weights) + cls4_biases
        logit_cls5 = tf.matmul(hidden, cls5_weights) + cls5_biases
        return logit_cls1, logit_cls2, logit_cls3, logit_cls4, logit_cls5
    
    logit_cls1, logit_cls2, logit_cls3, logit_cls4, logit_cls5 = model(tf_train_dataset, isTrain=True)
    
    def getLoss(logit, label):
        return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logit, label))
    
    loss = getLoss(logit_cls1, tf_train_labels[:,1]) + getLoss(logit_cls2, tf_train_labels[:,2]) +\
        getLoss(logit_cls3, tf_train_labels[:,3]) + getLoss(logit_cls4, tf_train_labels[:,4]) +\
        getLoss(logit_cls5, tf_train_labels[:,5])
    
    # Optimizer.
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(0.001, global_step, 10000, 0.9) 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Prediction for training
    train_prediction = tf.pack([tf.nn.softmax(logit_cls1),
                                tf.nn.softmax(logit_cls2),
                                tf.nn.softmax(logit_cls3), 
                                tf.nn.softmax(logit_cls4), 
                                tf.nn.softmax(logit_cls5)])

    # Prediction for valid
    v_logit_cls1, v_logit_cls2, v_logit_cls3, v_logit_cls4, v_logit_cls5 = model(tf_valid_dataset)
    valid_prediction = tf.pack([tf.nn.softmax(v_logit_cls1),
                                tf.nn.softmax(v_logit_cls2),
                                tf.nn.softmax(v_logit_cls3),
                                tf.nn.softmax(v_logit_cls4),
                                tf.nn.softmax(v_logit_cls5)])
    # Prediction for test
    t_logit_cls1, t_logit_cls2, t_logit_cls3, t_logit_cls4, t_logit_cls5 = model(tf_test_dataset)
    test_prediction = tf.pack([tf.nn.softmax(t_logit_cls1), 
                               tf.nn.softmax(t_logit_cls2), 
                               tf.nn.softmax(t_logit_cls3), 
                               tf.nn.softmax(t_logit_cls4), 
                               tf.nn.softmax(t_logit_cls5)])    
    
    # Init saver
    saver = tf.train.Saver()
    print(train_prediction)
    

Tensor("pack:0", shape=(5, 64, 11), dtype=float32)


In [95]:
save_path = "SVHN_MODEL.ckpt"
num_steps = 10000

with tf.Session(graph=graph) as session:

    tf.initialize_all_variables().run()

    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 500 == 0): 

            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels[:,1:6]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels[:,1:6]))
            print(predictions)
            break
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:6]))
    saved = saver.save(session, save_path)
    print("Model saved in file: %s" % saved)
    

Initialized
Minibatch loss at step 0: 24.816051
Minibatch accuracy: 8.1%


KeyboardInterrupt: 

In [93]:
save_path = "SVHN_MODEL.ckpt"

with tf.Session(graph=graph) as session:
    saver.restore(session, "SVHN_MODEL.ckpt")
    predict_prediction = session.run(valid_prediction, feed_dict={tf_train_dataset: train_dataset[:batch_size]})
    print(predict_prediction)[]

[[[  1.00263875e-04   5.65894833e-03   1.95526220e-02 ...,   2.67071021e-03
     8.65365437e-04   9.67706919e-01]
  [  1.31708017e-04   2.66676815e-03   9.53477994e-03 ...,   2.03006323e-02
     1.27777030e-05   9.31595981e-01]
  [  7.52819833e-05   4.52948269e-03   1.50673511e-03 ...,   3.94406170e-03
     8.02033173e-05   9.78980660e-01]
  ..., 
  [  3.81913758e-03   3.83738689e-02   7.20241526e-03 ...,   9.87094715e-02
     7.84072268e-04   6.10554099e-01]
  [  4.16698225e-04   1.23919714e-02   2.95306114e-03 ...,   6.21781033e-03
     1.04614859e-03   9.71646488e-01]
  [  1.16109470e-04   8.70383158e-03   1.88543685e-02 ...,   3.65398242e-03
     1.20819069e-03   9.59482729e-01]]

 [[  1.00263875e-04   5.65894833e-03   1.95526220e-02 ...,   2.67071021e-03
     8.65365437e-04   9.67706919e-01]
  [  1.31708017e-04   2.66676815e-03   9.53477994e-03 ...,   2.03006323e-02
     1.27777030e-05   9.31595981e-01]
  [  7.52819833e-05   4.52948269e-03   1.50673511e-03 ...,   3.94406170e-03
  